In [4]:
# Install Dependencies
!pip install --quiet transformers diffusers accelerate safetensors sentencepiece streamlit pyngrok==7.0.0 pdfplumber python-docx bitsandbytes Pillow > /dev/null

In [5]:
# Imports
import os, time, tempfile, sys, io
from pathlib import Path
from textwrap import shorten
from PIL import Image
import torch

In [6]:
# Model loading helper with fallback strategy
from diffusers import DiffusionPipeline
def try_load_sdxl(device):
    """Try loading SDXL base (best quality). If fails, raise."""
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    print("Attempting to load SDXL base model (best quality). This can take ~20-60s on GPU...")
    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
    )
    # Perf opts
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception as e:
        print("xformers not available or failed to enable:", e)
    return pipe
def try_load_sd2(device):
    """Fallback lightweight model (SD 2.1) for constrained runtimes."""
    model_id = "stabilityai/stable-diffusion-2-1-base"
    print("Loading Stable Diffusion 2.1 Base (fallback).")
    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
    )
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception:
        pass
    return pipe

In [7]:
# Load pipeline
import torch
from diffusers import DiffusionPipeline
# GPU Check
def check_gpu():
    gpu = torch.cuda.is_available()
    if gpu:
        try:
            !nvidia-smi
        except:
            pass
    return gpu
GPU_AVAILABLE = check_gpu()
print("GPU available:", GPU_AVAILABLE)
device = "cuda" if GPU_AVAILABLE else "cpu"
pipe = None
loaded_model_name = None
print("🔍 Selecting image generation model...")
# Public Model
def try_load_absolute_reality():
    print("🟢 Loading AbsoluteReality v1.8.1 (public, fast, realistic)...")
    return DiffusionPipeline.from_pretrained(
        "digiplay/AbsoluteReality_v1.8.1",
        torch_dtype=torch.float16 if GPU_AVAILABLE else torch.float32,
        use_safetensors=True
    )
# Load Model
try:
    pipe = try_load_absolute_reality()
    loaded_model_name = "AbsoluteReality_v1.8.1"
except Exception as e:
    print("❌ Failed to load AbsoluteReality:", e)
    raise RuntimeError("🚫 No image generation model could be loaded.")
# Move Model To GPU
if GPU_AVAILABLE:
    try:
        pipe = pipe.to("cuda")
        print("🚀 Model moved to GPU successfully.")
    except Exception as e:
        print("⚠️ GPU move failed:", e)
        print("➡ Keeping model on CPU (slower).")
else:
    print("⚠ No GPU available — running on CPU (very slow).")
print(f"✅ Loaded model: {loaded_model_name}  |  Device: {device}")

Thu Dec  4 18:31:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             25W /   70W |    2210MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

🚀 Model moved to GPU successfully.
✅ Loaded model: AbsoluteReality_v1.8.1  |  Device: cuda


In [8]:
# Summarization and character-extraction model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
SUM_MODEL = "google/flan-t5-base"  # fast & accurate for summarization; smaller & quicker than flan-t5-large
print("Loading summarization model:", SUM_MODEL)
tok = AutoTokenizer.from_pretrained(SUM_MODEL)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(SUM_MODEL).to(device if device=="cuda" else "cpu")
print("Summarization model loaded.")

Loading summarization model: google/flan-t5-base


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Summarization model loaded.


In [9]:
# Text extraction helpers (txt, pdf, docx)
import pdfplumber, docx
def extract_text_from_uploaded(file_obj, filename):
    """file_obj: BytesIO or UploadedFile-like; filename: name with extension"""
    ext = filename.split(".")[-1].lower()
    if ext == "txt":
        data = file_obj.read()
        if isinstance(data, bytes):
            return data.decode(errors="ignore")
        return str(data)
    elif ext == "pdf":
        with pdfplumber.open(file_obj) as pdf:
            pages = []
            for p in pdf.pages:
                text = p.extract_text()
                if text:
                    pages.append(text)
            return "\n\n".join(pages)
    elif ext == "docx":
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".docx")
        tmp.write(file_obj.read())
        tmp.flush(); tmp.close()
        doc = docx.Document(tmp.name)
        paragraphs = [p.text for p in doc.paragraphs if p.text.strip() != ""]
        os.unlink(tmp.name)
        return "\n\n".join(paragraphs)
    else:
        return ""

In [10]:
# Summarize & extract characters
def summarize_article(text, max_length=200):
    prompt = f"Summarize the following article into 5 concise scene descriptions (short sentences). Keep each scene to one line:\n\n{text}"
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model_summ.device)
    outputs = model_summ.generate(**inputs, max_length=max_length, num_beams=4, early_stopping=True)
    summary = tok.decode(outputs[0], skip_special_tokens=True)
    return summary
def extract_characters(text):
    prompt = f"Extract the main characters from the article and give 1-2 short physical descriptors and roles (e.g., 'young woman, determined activist') as a comma-separated list:\n\n{text}"
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model_summ.device)
    outputs = model_summ.generate(**inputs, max_length=150, num_beams=3, early_stopping=True)
    chars = tok.decode(outputs[0], skip_special_tokens=True)
    return chars
# Prompt builder - create cinematic, high-detail prompts with face consistency hinting
def build_prompts_from_summary(summary_text, characters, max_frames=6):
    scenes = []
    if "\n" in summary_text:
        cand = [s.strip() for s in summary_text.split("\n") if s.strip()]
    else:
        cand = [s.strip() for s in summary_text.split(".") if s.strip()]
    for s in cand:
        if len(s) >= 20:
            scenes.append(s)
        if len(scenes) >= max_frames:
            break
    if not scenes:
        scenes = [shorten(summary_text, width=120)]
    prompts = []
    for idx, scene in enumerate(scenes):
        prompt = (
            f"Ultra-detailed photorealistic photograph. {scene}. "
            f"Include characters: {characters}. "
            "Cinematic framing, filmic lighting, shallow depth of field, ultra sharp details, 8k photographic realism, award-winning photo, realistic skin tones, natural shadows."
        )
        prompts.append(prompt)
    return prompts

In [12]:
# Image generation wrapper with safety and fallback
import os
BASE_SAVE_DIR = "/content/drive/MyDrive/Article_Image_Generator"
os.makedirs(BASE_SAVE_DIR, exist_ok=True)
from PIL import Image
def generate_images_for_prompts(prompts, article_name="article", width=1024, height=1024, steps=28, save_dir=BASE_SAVE_DIR):
    out_paths = []
    safe_save_dir = os.path.join(save_dir, article_name.replace(" ", "_"))
    os.makedirs(safe_save_dir, exist_ok=True)
    for i, p in enumerate(prompts):
        try:
            if device == "cuda":
                img = pipe(prompt=p, height=height, width=width, num_inference_steps=steps).images[0]
            else:
                print("Warning: Generating on CPU. This will be slow.")
                img = pipe(prompt=p, height=768, width=768, num_inference_steps=20).images[0]
            filename = os.path.join(safe_save_dir, f"frame_{i+1}.png")
            img.save(filename)
            out_paths.append(filename)
            print(f"Saved {filename}")
        except Exception as e:
            print("Image generation error for prompt:", e)
            try:
                img = pipe(prompt=p, height=768, width=768, num_inference_steps=18).images[0]
                filename = os.path.join(safe_save_dir, f"frame_{i+1}_fallback.png")
                img.save(filename)
                out_paths.append(filename)
            except Exception as e2:
                print("Fallback generation failed:", e2)
    return out_paths

In [22]:
# Streamlit Backend
MAIN_PY = r"""
import os
import pdfplumber
import docx
import torch
from diffusers import DiffusionPipeline
from pathlib import Path
from PIL import Image
# -------------------------------
# CONFIG
# -------------------------------
BASE_SAVE_DIR = "/content/drive/MyDrive/Article_Image_Generator"
os.makedirs(BASE_SAVE_DIR, exist_ok=True)
# -------------------------------
# TEXT EXTRACTION
# -------------------------------
def extract_text_from_uploaded(file_obj, filename):
    ext = filename.split(".")[-1].lower()
    if ext == "txt":
        data = file_obj.read()
        return data.decode(errors="ignore")
    elif ext == "pdf":
        with pdfplumber.open(file_obj) as pdf:
            texts = [p.extract_text() for p in pdf.pages if p.extract_text()]
        return "\n\n".join(texts)
    elif ext == "docx":
        tmp_path = "/content/tmp.docx"
        with open(tmp_path, "wb") as f:
            f.write(file_obj.read())
        doc = docx.Document(tmp_path)
        return "\n".join([p.text for p in doc.paragraphs])
    else:
        return ""
# -------------------------------
# SUMMARIZATION
# -------------------------------
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
SUM_MODEL = "google/flan-t5-base"
tok = AutoTokenizer.from_pretrained(SUM_MODEL)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(SUM_MODEL).to("cuda")
def summarize_article(text):
    prompt = "Summarize this into 5 short scene descriptions:\n" + text
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
    outputs = model_summ.generate(inputs.input_ids, max_length=250, num_beams=4)
    return tok.decode(outputs[0], skip_special_tokens=True)
def extract_characters(text):
    prompt = "List main characters with short physical descriptions:\n" + text
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
    outputs = model_summ.generate(inputs.input_ids, max_length=120, num_beams=3)
    return tok.decode(outputs[0], skip_special_tokens=True)
# -------------------------------
# PROMPT BUILDER
# -------------------------------
def build_prompts_from_summary(summary, characters, max_frames=6):
    lines = [s.strip() for s in summary.split("\n") if len(s.strip()) > 10]
    lines = lines[:max_frames]
    prompts = []
    for s in lines:
        p = (
            f"Ultra-photorealistic cinematic image: {s}. "
            f"Characters: {characters}. "
            "8k detail, sharp focus, dramatic lighting, film still, award-winning photography."
        )
        prompts.append(p)
    return prompts
# -------------------------------
# IMAGE GENERATION
# -------------------------------
MODEL_ID = "digiplay/AbsoluteReality_v1.8.1"  # fast public model
pipe = DiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16
).to("cuda")
def generate_images_for_prompts(prompts, article_name):
    save_dir = os.path.join(BASE_SAVE_DIR, article_name.replace(" ", "_"))
    os.makedirs(save_dir, exist_ok=True)
    out_paths = []
    for i, prompt in enumerate(prompts):
        img = pipe(prompt=prompt, height=768, width=768, num_inference_steps=25).images[0]
        path = os.path.join(save_dir, f"frame_{i+1}.png")
        img.save(path)
        out_paths.append(path)
    return out_paths
"""
with open("main.py", "w") as f:
    f.write(MAIN_PY)
print("main.py created successfully.")

main.py created successfully.


In [23]:
# Stramlit UI
STREAMLIT_APP = r'''
import streamlit as st
import os
# Inject Custom CSS
st.markdown("""
<style>
body {
    background: linear-gradient(135deg, #1e3c72, #2a5298);
    color: white !important;
}
.main {
    background: transparent !important;
}
.block-container {
    background: rgba(255, 255, 255, 0.1);
    padding: 30px;
    border-radius: 20px;
    backdrop-filter: blur(8px);
    -webkit-backdrop-filter: blur(8px);
}
h1, h2, h3, h4, h5, h6 {
    color: #f5f7fa !important;
    text-shadow: 0px 1px 3px rgba(0,0,0,0.4);
}
.stButton>button {
    background-color: #ff7e5f;
    color: white !important;
    border-radius: 10px;
    padding: 0.6rem 1.2rem;
    font-weight: bold;
    border: none;
    transition: 0.3s;
}
.stButton>button:hover {
    background-color: #feb47b;
    transform: scale(1.05);
}
.uploadedFile {
    color: white !important;
}
</style>
""", unsafe_allow_html=True)
st.title("🌈 AI Article → Image Generator")
st.markdown("### Transform Entire Articles into Beautiful High-Resolution Images 📸✨")
mode = st.radio("Choose Mode", ["Single Article", "Batch Mode"])
from main import extract_text_from_uploaded, summarize_article, extract_characters, build_prompts_from_summary, generate_images_for_prompts
if mode == "Single Article":
    file = st.file_uploader("📄 Upload Article File", type=["txt", "pdf", "docx"])
    if file:
        with st.spinner("Reading file..."):
            text = extract_text_from_uploaded(file, file.name)
        st.subheader("📘 Article Preview")
        st.write(text[:1000] + ("..." if len(text) > 1000 else ""))
        if st.button("Generate Images"):
            with st.spinner("Summarizing article..."):
                summary = summarize_article(text)
                characters = extract_characters(text)
                prompts = build_prompts_from_summary(summary, characters)
            st.subheader("🧠 Generated Prompts")
            for i, p in enumerate(prompts):
                st.markdown(f"**Frame {i+1}:** {p}")
            with st.spinner("Creating images..."):
                images = generate_images_for_prompts(prompts, article_name=file.name.split('.')[0])
            st.success("🎉 Image Generation Complete!")
            for img_path in images:
                st.image(img_path, width=450)
else:
    files = st.file_uploader("📄 Upload Multiple Files", type=["txt","pdf","docx"], accept_multiple_files=True)
    if files and st.button("Process All"):
        for file in files:
            st.write(f"➡ Processing **{file.name}**...")
            text = extract_text_from_uploaded(file, file.name)
            summary = summarize_article(text)
            characters = extract_characters(text)
            prompts = build_prompts_from_summary(summary, characters)
            images = generate_images_for_prompts(prompts, article_name=file.name.split('.')[0])
        st.success("🎉 Batch Processing Complete! All images saved to Google Drive.")
'''
with open("app.py", "w") as f:
    f.write(STREAMLIT_APP)
print("app.py created successfully.")

app.py created successfully.


In [24]:
# Inject the real functions into the main.py module so Streamlit can import them
import types
import sys
# Create a dynamic module named "main"
main_mod = types.ModuleType("main")
# Assign actual functions & variables from notebook
main_mod.extract_text_from_uploaded = extract_text_from_uploaded
main_mod.summarize_article = summarize_article
main_mod.extract_characters = extract_characters
main_mod.build_prompts_from_summary = build_prompts_from_summary
main_mod.generate_images_for_prompts = generate_images_for_prompts
main_mod.BASE_SAVE_DIR = BASE_SAVE_DIR
# Register the module so that `from main import *` works inside Streamlit
sys.modules["main"] = main_mod
print("✅ Successfully injected notebook functions into 'main' module.")
print("🎉 Streamlit app is now ready!")

✅ Successfully injected notebook functions into 'main' module.
🎉 Streamlit app is now ready!


In [25]:
# Streamlit App Deployment
!pip install -q streamlit pyngrok
import os
import time
import subprocess
from pyngrok import ngrok, conf
import logging
logging.getLogger("pyngrok.process").setLevel(logging.ERROR)
# Configure Ngrok Token
NGROK_AUTH_TOKEN = "2z0Oqv0tD166fELGCHwV2gLZwq1_2G2zUQRSs6C27k9vdzxwq"
conf.get_default().auth_token = NGROK_AUTH_TOKEN
# Prepare log directory
LOG_DIR = "/content/logs"
os.makedirs(LOG_DIR, exist_ok=True)
# Clean any running instance
!pkill -f streamlit 2>/dev/null || true
ngrok.kill()
print("⚙️ Starting Streamlit...")
# Launch Streamlit server
streamlit_cmd = [
    "streamlit", "run", "app.py",
    "--server.port", "8501",
    "--server.address", "0.0.0.0"
]
log_file_path = f"{LOG_DIR}/app_log.txt"
with open(log_file_path, "w") as log_file:
    process = subprocess.Popen(
        streamlit_cmd,
        stdout=log_file,
        stderr=log_file,
        text=True
    )
time.sleep(8)
# Create Ngrok tunnel
public_url = ngrok.connect(addr=8501)
print("🚀 Your Streamlit app is live at:", public_url)
print("📜 Logs at:", log_file_path)

^C
⚙️ Starting Streamlit...
🚀 Your Streamlit app is live at: NgrokTunnel: "https://8de659161fe9.ngrok-free.app" -> "http://localhost:8501"
📜 Logs at: /content/logs/app_log.txt
